# Regression

In [1]:
import pandas as pd
import numpy as np
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
genomes = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="genomics_cannabis")

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "genomics_cannabis")
bq_assistant.list_tables()

['MNPR01_201703',
 'MNPR01_reference_201703',
 'MNPR01_transcriptome_201703',
 'cs10_gff',
 'cs3k_project_info',
 'cs3k_vcf_cs10_dv090',
 'sample_info_201703']

In [3]:
query = """SELECT
  variant_id, alternate_bases, quality, type, 
  AB, ABP, AC, AF, AN, AO, DP, DPB, DPRA, EPP, 
  EPPR, GTI, MQM, MQMR, NS, NUMALT, ODDS, PAO,
  PQA, PQR, PRO, QA, QR, RO, RPL, RPP, RPPR, 
  RPR, RUN, SAF, SAP, SAR, SRF, SRP, SRR
FROM
  `bigquery-public-data.genomics_cannabis.MNPR01_201703` v
limit 1000000;"""
response = genomes.query_to_pandas_safe(query, max_gb_scanned=100)
response.head(10)

C:\Users\sharm\AppData\Roaming\Python\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,variant_id,alternate_bases,quality,type,AB,ABP,AC,AF,AN,AO,...,RPP,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR
0,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTI1NzF8Z2J8TU5QUj...,[CTA],23.39620,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
1,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTEyODV8Z2J8TU5QUj...,[CGTAA],7.52241,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
2,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY5MTJ8Z2J8TU5QUj...,[CCTT],25.30020,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
3,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY5OTl8Z2J8TU5QUj...,[TATGG],6.16805,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
4,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODY2OTF8Z2J8TU5QUj...,[ATTTTTTTAA],3.08123,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
5,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTQyOTF8Z2J8TU5QUj...,[GAG],23.56570,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
6,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTUwNTV8Z2J8TU5QUj...,[GC],27.00690,[del],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
7,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTUyMDF8Z2J8TU5QUj...,[GATAAA],14.01670,[mnp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
8,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU1NTV8Z2J8TU5QUj...,[TTTTGC],12.62600,[mnp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
9,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODgxOTZ8Z2J8TU5QUj...,[AGT],9.12466,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[0],[5.18177],[1],0,0.0,0


I shall run regression to determine my DNA samples based on proteins. 

In [4]:
response["DNA_A"] = response["alternate_bases"].str[0].str.count('A')

In [5]:
response["DNA_C"] = response["alternate_bases"].str[0].str.count('C')

In [6]:
response["DNA_G"] = response["alternate_bases"].str[0].str.count('G')

In [7]:
response["DNA_T"] = response["alternate_bases"].str[0].str.count('T')

Now I shall adjust the proteins for easy analysis. 

In [8]:
proteins = ["AB", "ABP", "AC", "AF", "AO", "DPRA", "EPP", "MQM", "PAO",
            "PQA", "QA", "RPL", "RPP", 
            "RPR", "RUN", "SAF", "SAP", "SAR", 
]

for protein in proteins:
    print(protein)
    response[protein] = response[protein].str[0]

AB
ABP
AC
AF
AO
DPRA
EPP
MQM
PAO
PQA
QA
RPL
RPP
RPR
RUN
SAF
SAP
SAR


In [9]:
response["type"] = response["type"].str[0]

In [10]:
response["type"].value_counts()

type
snp        699662
complex    158031
mnp         57898
ins         51333
del         33076
Name: count, dtype: int64

```type``` is a categorical variable, so I shall need to dummify it. 

In [11]:
dummies = pd.get_dummies(response["type"], dtype="int")
dummies

,complex,del,ins,mnp,snp
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,0,1
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
999995,0,0,0,0,1
999996,1,0,0,0,0
999997,0,0,0,0,1
999998,1,0,0,0,0


Now I shall define what I am regressing upon.

In [12]:
y = response[["DNA_A", "DNA_C", "DNA_G", "DNA_T"]]

In [13]:
X = pd.concat([
    response[["AB", "ABP", "AC", "AF", "AN", "AO", "DP", "DPB", "DPRA", "EPP", 
              "EPPR", "GTI", "MQM", "MQMR", "NS", "NUMALT", "ODDS", "PAO",
              "PQA", "PQR", "PRO", "QA", "QR", "RO", "RPL", "RPP", "RPPR", 
              "RPR", "RUN", "SAF", "SAP", "SAR", "SRF", "SRP", "SRR"
    ]], 
    dummies
], axis=1)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=1337)
LR = LinearRegression()
LR.fit(X_train, y_train)
train_score = LR.score(X_train, y_train)
test_score = LR.score(X_test, y_test)
print(f'Train score {train_score}, test score {test_score}')

Train score 0.17432333633747393, test score 0.15232295290835307


I've achieved a test score of 15.2%.

In [20]:
X.columns

Index(['AB', 'ABP', 'AC', 'AF', 'AN', 'AO', 'DP', 'DPB', 'DPRA', 'EPP', 'EPPR',
       'GTI', 'MQM', 'MQMR', 'NS', 'NUMALT', 'ODDS', 'PAO', 'PQA', 'PQR',
       'PRO', 'QA', 'QR', 'RO', 'RPL', 'RPP', 'RPPR', 'RPR', 'RUN', 'SAF',
       'SAP', 'SAR', 'SRF', 'SRP', 'SRR', 'complex', 'del', 'ins', 'mnp',
       'snp'],
      dtype='object')

In [19]:
LR.coef_

array([[ 9.41268071e-02, -1.61469767e-03, -2.32882588e+06,
         4.65765182e+06, -1.36292671e+06, -1.83319931e+08,
        -1.97445380e-03,  6.29906730e-03, -6.57372037e+05,
         5.96583000e-04, -6.33063850e-04,  1.70826157e-01,
         2.90752616e-04, -9.14484815e-04,  1.12762499e+04,
        -7.84086359e-03, -1.01120586e-03, -1.13337899e-02,
         1.66021188e-03, -4.38054265e-04,  2.95612977e-03,
         7.21622241e-05,  1.05638228e-04,  8.22346880e+06,
         9.22203436e+07, -1.67460235e-03, -3.11152659e-04,
         9.22203436e+07,  4.41608849e+02,  9.10995870e+07,
         1.30512456e-03,  9.10995871e+07, -8.22346880e+06,
        -1.71927298e-03, -8.22346881e+06, -6.03171995e+04,
        -6.03172462e+04, -6.03167089e+04, -6.03183080e+04,
        -6.03188022e+04],
       [-1.83488950e-01, -2.26307330e-03, -7.29279202e+06,
         1.45855840e+07, -4.26804813e+06, -5.74072164e+08,
         1.29853571e-04,  2.66689298e-03, -2.05858134e+06,
         9.98401378e-04, -7.88

In [24]:
coeffs = pd.DataFrame(data=LR.coef_, columns=X.columns, index=y.columns)

In [26]:
coeffs.transpose()

,DNA_A,DNA_C,DNA_G,DNA_T
AB,9.412681e-02,-1.834889e-01,-1.815812e-01,5.469601e-02
ABP,-1.614698e-03,-2.263073e-03,-1.976247e-03,2.019088e-03
AC,-2.328826e+06,-7.292792e+06,1.549000e+06,9.457202e+06
AF,4.657652e+06,1.458558e+07,-3.098001e+06,-1.891440e+07
AN,-1.362927e+06,-4.268048e+06,9.065401e+05,5.534751e+06
AO,-1.833199e+08,-5.740722e+08,1.219338e+08,7.444496e+08
DP,-1.974454e-03,1.298536e-04,1.177305e-03,6.392662e-03
DPB,6.299067e-03,2.666893e-03,2.744768e-03,6.000936e-03
DPRA,-6.573720e+05,-2.058581e+06,4.372459e+05,2.669543e+06
EPP,5.965830e-04,9.984014e-04,9.222286e-04,-4.385499e-04


The strongest coefficients are the proteins AO, RPL, RPR, SAF, and SAR. 

Notably, there is also a strong correlation between all four of the DNA types in terms of protein coefficients between them. 